In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="BreastCancer"
JobNameAbbrev = "BC"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.05],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["11:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["30000M"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Random Forest Simulations

In [3]:
# ### Include Random Forest ###
# RandomForestParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["TreeEnsembleQBCFunction"],
#                        "ModelType":["RandomForestClassificationFunction"],
#                        "UniqueErrorsInput": [0],
#                        "n_estimators": [100], 
#                        "regularization": [0.01],
#                        "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
#                        "RashomonThreshold": [0],
#                        "Type": ["Classification"],
#                        "Partition": ["compute"],                                                        # [short, medium, long, largemem, or compute]
#                        "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
#                        "Memory": [1000]}                                                                # [1000, 30000, 100000]

# RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# # NOTE: Comment out chunk to not include random forest simulations. ###
# ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
# ParameterVector = ParameterVector.sort_values("Seed")
# ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [4]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True))

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

In [5]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,Partition,Time,Memory,JobName,Output
0,BreastCancer,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,0BC_DA0.05,BreastCancer/TreeFarms/Raw/0BC_DA0.05.pkl
1,BreastCancer,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,0BC_UA0.05,BreastCancer/TreeFarms/Raw/0BC_UA0.05.pkl
2,BreastCancer,1,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,1BC_DA0.05,BreastCancer/TreeFarms/Raw/1BC_DA0.05.pkl
3,BreastCancer,1,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,1BC_UA0.05,BreastCancer/TreeFarms/Raw/1BC_UA0.05.pkl
4,BreastCancer,2,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,2BC_DA0.05,BreastCancer/TreeFarms/Raw/2BC_DA0.05.pkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,BreastCancer,97,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,97BC_UA0.05,BreastCancer/TreeFarms/Raw/97BC_UA0.05.pkl
196,BreastCancer,98,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,98BC_DA0.05,BreastCancer/TreeFarms/Raw/98BC_DA0.05.pkl
197,BreastCancer,98,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,98BC_UA0.05,BreastCancer/TreeFarms/Raw/98BC_UA0.05.pkl
198,BreastCancer,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,99BC_DA0.05,BreastCancer/TreeFarms/Raw/99BC_DA0.05.pkl


---

# Only run these ones

In [6]:
# ### Simulations that failed ###
FilterText = ['26BC_DA0.05', '27BC_DA0.05', '28BC_DA0.05', '29BC_DA0.05', '30BC_DA0.05', '32BC_DA0.05', '33BC_DA0.05', '34BC_DA0.05', '36BC_DA0.05', '39BC_DA0.05', '40BC_DA0.05', '41BC_DA0.05', '42BC_DA0.05', '43BC_DA0.05', '44BC_DA0.05', '45BC_DA0.05', '46BC_DA0.05', '47BC_DA0.05', '48BC_DA0.05', '49BC_DA0.05', '26BC_UA0.05', '27BC_UA0.05', '28BC_UA0.05', '29BC_UA0.05', '30BC_UA0.05', '32BC_UA0.05', '33BC_UA0.05', '34BC_UA0.05', '36BC_UA0.05', '39BC_UA0.05', '40BC_UA0.05', '41BC_UA0.05', '42BC_UA0.05', '43BC_UA0.05', '44BC_UA0.05', '45BC_UA0.05', '46BC_UA0.05', '47BC_UA0.05', '48BC_UA0.05', '49BC_UA0.05']
### Rerun these simulations ###
ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
ParameterVector.index = range(0,len(ParameterVector))
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,Partition,Time,Memory,JobName,Output
0,BreastCancer,26,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,26BC_DA0.05,BreastCancer/TreeFarms/Raw/26BC_DA0.05.pkl
1,BreastCancer,26,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,26BC_UA0.05,BreastCancer/TreeFarms/Raw/26BC_UA0.05.pkl
2,BreastCancer,27,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,27BC_DA0.05,BreastCancer/TreeFarms/Raw/27BC_DA0.05.pkl
3,BreastCancer,27,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,27BC_UA0.05,BreastCancer/TreeFarms/Raw/27BC_UA0.05.pkl
4,BreastCancer,28,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,28BC_DA0.05,BreastCancer/TreeFarms/Raw/28BC_DA0.05.pkl
5,BreastCancer,28,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,28BC_UA0.05,BreastCancer/TreeFarms/Raw/28BC_UA0.05.pkl
6,BreastCancer,29,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,29BC_DA0.05,BreastCancer/TreeFarms/Raw/29BC_DA0.05.pkl
7,BreastCancer,29,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,29BC_UA0.05,BreastCancer/TreeFarms/Raw/29BC_UA0.05.pkl
8,BreastCancer,30,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,30BC_DA0.05,BreastCancer/TreeFarms/Raw/30BC_DA0.05.pkl
9,BreastCancer,30,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,short,11:59:00,30000M,30BC_UA0.05,BreastCancer/TreeFarms/Raw/30BC_UA0.05.pkl


---

# Save Parameter Vector

In [7]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)